In [1]:
!pip install MLB-StatsAPI

In [31]:
import statsapi
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import pymysql
from sqlalchemy import create_engine

In [4]:
print(statsapi.boxscore(565997))

------------------------------------------------------------------------------- | -------------------------------------------------------------------------------
Phillies Batters                         AB   R   H  RBI BB   K  LOB AVG   OPS  | Mets Batters                             AB   R   H  RBI BB   K  LOB AVG   OPS 
------------------------------------------------------------------------------- | -------------------------------------------------------------------------------
1 McCutchen  LF                           5   0   1   0   0   1   3  .250 .830  | 1 McNeil  LF                              4   0   1   0   0   0   1  .363 .928 
2 Realmuto  C                             3   1   1   0   1   1   2  .282 .786  | 2 Conforto  RF                            3   0   0   0   1   1   1  .292 .986 
3 Harper  RF                              4   1   1   1   1   3   4  .261 .909  | 3 Canó, R  2B                             3   0   3   0   1   0   0  .272 .758 
4 Hoskins  1B               

In [8]:
statsapi.standings(date='06/22/2024')

'American League East\nRank Team                   W   L   GB  (E#) WC Rank WC GB (E#)\n 1   New York Yankees      51  27   -    -      -      -    -  \n 2   Baltimore Orioles     49  26  0.5   86     1    +8.5   -  \n 3   Boston Red Sox        40  36  10.0  76     4     1.0   86 \n 4   Tampa Bay Rays        37  39  13.0  73     5     4.0   83 \n 5   Toronto Blue Jays     35  40  14.5  72     8     5.5   82 \n\nAmerican League Central\nRank Team                   W   L   GB  (E#) WC Rank WC GB (E#)\n 1   Cleveland Guardians   47  26   -    -      -      -    -  \n 2   Kansas City Royals    42  35  7.0   81     2    +0.5   -  \n 3   Minnesota Twins       41  35  7.5   81     3      -    -  \n 4   Detroit Tigers        35  41  13.5  75     9     6.0   81 \n 5   Chicago White Sox     21  57  28.5  59    12    21.0   65 \n\nAmerican League West\nRank Team                   W   L   GB  (E#) WC Rank WC GB (E#)\n 1   Seattle Mariners      44  34   -    -      -      -    -  \n 2   Houston Ast

In [9]:
print(statsapi.standings(date='06/22/2024'))

American League East
Rank Team                   W   L   GB  (E#) WC Rank WC GB (E#)
 1   New York Yankees      51  27   -    -      -      -    -  
 2   Baltimore Orioles     49  26  0.5   86     1    +8.5   -  
 3   Boston Red Sox        40  36  10.0  76     4     1.0   86 
 4   Tampa Bay Rays        37  39  13.0  73     5     4.0   83 
 5   Toronto Blue Jays     35  40  14.5  72     8     5.5   82 

American League Central
Rank Team                   W   L   GB  (E#) WC Rank WC GB (E#)
 1   Cleveland Guardians   47  26   -    -      -      -    -  
 2   Kansas City Royals    42  35  7.0   81     2    +0.5   -  
 3   Minnesota Twins       41  35  7.5   81     3      -    -  
 4   Detroit Tigers        35  41  13.5  75     9     6.0   81 
 5   Chicago White Sox     21  57  28.5  59    12    21.0   65 

American League West
Rank Team                   W   L   GB  (E#) WC Rank WC GB (E#)
 1   Seattle Mariners      44  34   -    -      -      -    -  
 2   Houston Astros        36  40  7

In [34]:
data = statsapi.standings(date='06/22/2024')
tables = re.split(r'\n\n', data.strip())

dfs, columns = {}, ['Rank', 'Team', 'W', 'L', 'GB', '(E#)', 'WC Rank', 'WC GB', 'WC (E#)']

for table in tables:
    lines = table.strip().split('\n')
    name = lines[0]
    data_lines = lines[2:]
    
    # Create a list of rows
    rows = []
    for line in data_lines:
        row = re.split(r'\s{2,}', line.strip())
        rows.append(row)
    
    # Create a DataFrame
    df = pd.DataFrame(rows, columns=columns)
    dfs[name] = df

# Display the DataFrames
for name, df in dfs.items():
    print(f"Table: {name}")
    display(df)
    print()

Table: American League East


,Rank,Team,W,L,GB,(E#),WC Rank,WC GB,WC (E#)
0,1,New York Yankees,52,27,-,-,-,-,-
1,2,Baltimore Orioles,49,27,1.5,84,1,+8.0,-
2,3,Boston Red Sox,41,36,10.0,75,4,0.5,85
3,4,Tampa Bay Rays,37,40,14.0,71,6,4.5,81
4,5,Toronto Blue Jays,35,41,15.5,70,9,6.0,80



Table: American League Central


,Rank,Team,W,L,GB,(E#),WC Rank,WC GB,WC (E#)
0,1,Cleveland Guardians,48,26,-,-,-,-,-
1,2,Minnesota Twins,42,35,7.5,80,2,+0.5,-
2,3,Kansas City Royals,42,36,8.0,79,3,-,-
3,4,Detroit Tigers,35,41,14.0,74,8,6.0,80
4,5,Chicago White Sox,21,57,29.0,58,12,21.0,64



Table: American League West


...

,Rank,Team,W,L,GB,(E#),WC Rank,WC GB,WC (E#)
0,1,Philadelphia Phillies 50,26,-,-,-,-,-,None
1,2,Atlanta Braves,42,32,7.0,81,1,+4.5,-
2,3,Washington Nationals,37,39,13.0,74,6,1.5,86
3,4,New York Mets,36,39,13.5,74,8,2.0,86
4,5,Miami Marlins,26,50,24.0,63,12,12.5,75



Table: National League Central


,Rank,Team,W,L,GB,(E#),WC Rank,WC GB,WC (E#)
0,1,Milwaukee Brewers,44,33,-,-,-,-,-
1,2,St. Louis Cardinals,38,37,5.0,82,2,-,-
2,3,Pittsburgh Pirates,37,39,6.5,80,5,1.5,86
3,4,Chicago Cubs,37,40,7.0,79,7,2.0,85
4,5,Cincinnati Reds,36,40,7.5,79,9,2.5,85



Table: National League West


,Rank,Team,W,L,GB,(E#),WC Rank,WC GB,WC (E#)
0,1,Los Angeles Dodgers,48,31,-,-,-,-,-
1,2,San Diego Padres,41,40,8.0,75,3,-,-
2,3,Arizona Diamondbacks,38,39,9.0,76,4,1.0,86
3,4,San Francisco Giants,36,41,11.0,74,10,3.0,84
4,5,Colorado Rockies,27,50,20.0,65,11,12.0,75


In [37]:
conn = pymysql.connect(
    host='Atharvas-MacBook-Air.local',
    port=int(3306),
    user='root',
    passwd='mypassword',
    db='mlb-db',
    charset='utf8mb4')

cursor = conn.cursor()

In [71]:
dfs['National League West']

,Rank,Team,W,L,GB,(E#),WC Rank,WC GB,WC (E#)
0,1,Los Angeles Dodgers,48,31,-,-,-,-,-
1,2,San Diego Padres,41,40,8.0,75,3,-,-
2,3,Arizona Diamondbacks,38,39,9.0,76,4,1.0,86
3,4,San Francisco Giants,36,41,11.0,74,10,3.0,84
4,5,Colorado Rockies,27,50,20.0,65,11,12.0,75


In [32]:
engine = create_engine('mysql+pymysql://', creator=lambda: conn)

In [70]:
for tab, df in dfs.items():
    df.to_sql(name=tab, con=engine, if_exists='replace', index=False, method='multi')
    print(f"Table {tab} successfully written to the database.")

Table American League East successfully written to the database.
Table American League Central successfully written to the database.
Table American League West successfully written to the database.
Table National League East successfully written to the database.
Table National League Central successfully written to the database.
Table National League West successfully written to the database.


In [108]:
# Debug to show tables in mlb-db

cursor.execute("SHOW TABLES")
tables = [table[0] for table in cursor.fetchall()]
print([t for t in tables])

['mlb_standings']


In [109]:
for table in tables:
    # print(f"DROP TABLE `{table}`")
    cursor.execute(f"DROP TABLE `{table}`")
    print(f"Dropped table {table}")

conn.commit()

Dropped table mlb_standings


In [99]:
# Testing for function-approach

def fetch_strip_standings(date):
    data = statsapi.standings(date=date)
    tables = re.split(r'\n\n', data.strip())
    
    columns = ['Rank', 'Team', 'W', 'L', 'GB', '(E#)', 'WC Rank', 'WC GB', 'WC (E#)']
    dfs = {}
    
    for table in tables:
        lines = table.strip().split('\n')
        name = lines[0]
        data_lines = lines[2:]
        
        rows = []
        for line in data_lines:
            row = re.split(r'\s{2,}', line.strip())
            rows.append(row)
        
        df = pd.DataFrame(rows, columns=columns)
        dfs[name] = df
        
    return dfs

date = '06/22/2024'
standings_dfs = fetch_strip_standings(date)

for name, df in standings_dfs.items():
    print(f"Table: {name}")
    display(df)
    print()

Table: American League East


,Rank,Team,W,L,GB,(E#),WC Rank,WC GB,WC (E#)
0,1,New York Yankees,52,27,-,-,-,-,-
1,2,Baltimore Orioles,49,27,1.5,84,1,+8.0,-
2,3,Boston Red Sox,41,36,10.0,75,4,0.5,85
3,4,Tampa Bay Rays,37,40,14.0,71,6,4.5,81
4,5,Toronto Blue Jays,35,41,15.5,70,9,6.0,80



Table: American League Central


,Rank,Team,W,L,GB,(E#),WC Rank,WC GB,WC (E#)
0,1,Cleveland Guardians,48,26,-,-,-,-,-
1,2,Minnesota Twins,42,35,7.5,80,2,+0.5,-
2,3,Kansas City Royals,42,36,8.0,79,3,-,-
3,4,Detroit Tigers,35,41,14.0,74,8,6.0,80
4,5,Chicago White Sox,21,57,29.0,58,12,21.0,64



Table: American League West


...

,Rank,Team,W,L,GB,(E#),WC Rank,WC GB,WC (E#)
0,1,Philadelphia Phillies 50,26,-,-,-,-,-,None
1,2,Atlanta Braves,42,32,7.0,81,1,+4.5,-
2,3,Washington Nationals,37,39,13.0,74,6,1.5,86
3,4,New York Mets,36,39,13.5,74,8,2.0,86
4,5,Miami Marlins,26,50,24.0,63,12,12.5,75



Table: National League Central


,Rank,Team,W,L,GB,(E#),WC Rank,WC GB,WC (E#)
0,1,Milwaukee Brewers,44,33,-,-,-,-,-
1,2,St. Louis Cardinals,38,37,5.0,82,2,-,-
2,3,Pittsburgh Pirates,37,39,6.5,80,5,1.5,86
3,4,Chicago Cubs,37,40,7.0,79,7,2.0,85
4,5,Cincinnati Reds,36,40,7.5,79,9,2.5,85



Table: National League West


,Rank,Team,W,L,GB,(E#),WC Rank,WC GB,WC (E#)
0,1,Los Angeles Dodgers,48,31,-,-,-,-,-
1,2,San Diego Padres,41,40,8.0,75,3,-,-
2,3,Arizona Diamondbacks,38,39,9.0,76,4,1.0,86
3,4,San Francisco Giants,36,41,11.0,74,10,3.0,84
4,5,Colorado Rockies,27,50,20.0,65,11,12.0,75


In [110]:
# Creating table for time-series analysis of standings

teams = [
    'Orioles', 'Red_Sox', 'Yankees', 'Rays', 'Blue_Jays',  # AL East
    'White_Sox', 'Guardians', 'Tigers', 'Royals', 'Twins',  # AL Central
    'Astros', 'Angels', 'Athletics', 'Mariners', 'Rangers',  # AL West
    'Braves', 'Marlins', 'Mets', 'Phillies', 'Nationals',  # NL East
    'Cubs', 'Reds', 'Brewers', 'Pirates', 'Cardinals',  # NL Central
    'Diamondbacks', 'Rockies', 'Dodgers', 'Padres', 'Giants'  # NL West
]

create_tsTab = """
CREATE TABLE IF NOT EXISTS mlb_standings (
    Date DATE,
    {}
)
""".format(', '.join([f"`{team}` INT" for team in teams]))

cursor = conn.cursor()
cursor.execute(create_tsTab)
conn.commit()

In [107]:
from datetime import datetime

def insert_standings_into_db(date, standings_dfs):

    ranks = {'Date': datetime.strptime(date, '%m/%d/%Y').strftime('%Y-%m-%d')}
    for df in standings_dfs.values():
        for index, row in df.iterrows():
            team = row['Team']
            rank = row['Rank']
            
            team_nickname = team.split()[-1]
            if team_nickname == 'Sox':  # Handle both Red Sox and White Sox
                team_nickname = 'Red_Sox' if 'Red' in team else 'White_Sox'
            elif team_nickname == 'Jays':
                team_nickname = 'Blue_Jays'
            
            column_name = team_nickname
            ranks[column_name] = rank

    # Check and replace numeric column names with 'Phillies'
    for key in list(ranks.keys()):
        if key.isdigit():
            ranks['Phillies'] = ranks.pop(key)
    
    # Convert to DataFrame
    standings_df = pd.DataFrame([ranks])
    
    # Insert into database
    standings_df.to_sql('mlb_standings', engine, if_exists='append', index=False)

# Example usage
date = '06/22/2024'
standings_dfs = fetch_and_process_standings(date)
insert_standings_into_db(date, standings_dfs)

In [114]:
# Doing for every match-week

from datetime import datetime, timedelta

# Automate for every match week
start_date = datetime(2023, 4, 1)
end_date = datetime(2023, 10, 1)
date = start_date

while date <= end_date:
    date_str = date.strftime('%m/%d/%Y')
    standings_dfs = fetch_and_process_standings(date_str)
    insert_standings_into_db(date_str, standings_dfs)
    date += timedelta(weeks=1)